In [ ]:
import torch
import clip
from PIL import Image
import requests
from baukit import save_image_set
from baukit import ImageFolderSet, show, move_to, pbar
import torchvision, os
from torch.nn.functional import cosine_similarity
from torchvision.datasets.utils import download_and_extract_archive
torch.set_grad_enabled(False)

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

if not os.path.isdir('coco_humans'):
    download_and_extract_archive('https://cs7150.baulab.info/2022-Fall/data/coco_humans.zip', 'coco_humans')

images = ImageFolderSet('coco_humans')
image_features = torch.cat(
    [model.encode_image(preprocess(im[0])[None].to(device)) for im in pbar(images)]
)



In [ ]:
image_tensor = preprocess(images[0][0])
show(images[0][0], str(image_tensor.shape), image_tensor)


In [ ]:
text_tensor = clip.tokenize('a sunny day')

In [ ]:
model.encode_text(text_tensor.cuda())

In [ ]:
model.encode_image(image_tensor[None].cuda())

In [ ]:
from baukit import Textbox, Div

def do_search():
    tokens = clip.tokenize([query.value]).to(device)
    text_features = model.encode_text(tokens)
    sim = cosine_similarity(text_features, image_features, dim=1)
    result.show(show.WRAP, [[f'{rank+1}: {s:.3f}', images[i]] for rank, (s, i) in enumerate(zip(*sim.topk(12)))])
query, result = [Textbox().on('value', do_search), Div()]

show([show.attr(placeholder='enter your query'), query],
     result)

In [ ]:
tokens = clip.tokenize([query.value]).to(device)
text_features = model.encode_text(tokens)
sim = cosine_similarity(text_features, image_features, dim=1)
result.show(show.WRAP, [[rank+1, images[i]] for rank, (s, i) in enumerate(zip(*sim.topk(12)))])
query, result = [Textbox().on('value', do_search), Div()]


In [ ]:
list(zip(*sim.topk(12)))